In [258]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import urllib.parse

In [259]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [260]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [261]:
%%capture pwd
!pwd

In [262]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
uid=urllib.parse.quote(uid)
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('what-is-the-likely-impact-of-the-earthquakes-on-economic-growth-in-t%25C3%25BCrkiye',
 300,
 500)

# Fig 1

In [263]:
df = pd.read_excel("raw/Charts.xlsx", sheet_name='data2')

In [264]:
f = "fig1_earthquake"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,d,year,gdpt,gdps,govt,govs,side,gdpci,govci
0,NaN,1994,78.647739,79.598530,13.971999,14.615019,right,NaN,NaN
1,NaN,1995,84.303468,84.448010,13.359027,13.695638,right,NaN,NaN
2,NaN,1996,90.209196,90.310738,12.853271,12.698432,right,NaN,NaN
3,NaN,1997,97.000870,97.047676,13.211440,13.065418,right,NaN,NaN
4,NaN,1998,100.000000,100.000000,14.585665,14.451184,right,NaN,NaN


In [265]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "year:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
            format='.0f'
        ),
    )
)
line = base.mark_line(color=colors["eco-dot"]).encode(
    y=alt.Y(
        "gdpt:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="Real GDP (index, 1998=100)",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        scale=alt.Scale(domain=[70, 130]),
    )
)
line2=line.mark_line(color=colors["eco-mid-blue"]).encode(y='gdps:Q')
line3 = base.mark_line(color=colors["eco-dot"]).encode(
    y=alt.Y(
        "govt:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="Government spending as a share of GDP (%)",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        scale=alt.Scale(domain=[5, 25]),
    )
)
line4=line.mark_line(color=colors["eco-mid-blue"]).encode(y='govs:Q')
base = alt.Chart(f1).encode(
    x=alt.X(
        "year:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0
        ),
    )
)
base2=base.encode( x=alt.X(
        "d:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
            format='.0f',
            labelExpr="datum.label == 0 ? ' ' : datum.label > 0 ? 't+'+datum.label :'t'+datum.label"
        ),
    )
).transform_calculate(y1='datum.gdpt-datum.gdpci/1')\
.transform_calculate(y2='datum.gdpt+datum.gdpci/1')\
.transform_calculate(y3='datum.govt-datum.govci/1')\
.transform_calculate(y4='datum.govt+datum.govci/1')
area1=base2.mark_area(color=colors['eco-dot'],opacity=0.1).encode(
    y=alt.Y(
        "y1:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="Real GDP (index, 1998=100)",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"]
        ),
        scale=alt.Scale(domain=[70, 130]),
    ),
    y2='y2:Q'
)
line5 = base2.mark_line(color=colors["eco-dot"]).encode(
    y="gdpt:Q"
)
line6=line5.mark_line(color=colors["eco-mid-blue"]).encode(y='gdps:Q')
area2=base2.mark_area(color=colors['eco-dot'],opacity=0.1).encode(
    y=alt.Y(
        "y3:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="Government spending as a share of GDP (%)",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"]
        ),
        scale=alt.Scale(domain=[5, 25]),
    ),
    y2='y4:Q'
)
line7 = base2.mark_line(color=colors["eco-dot"]).encode(
    y="govt:Q"
)
line8=line7.mark_line(color=colors["eco-mid-blue"]).encode(y='govs:Q')
eq=base.mark_rule(color=colors['eco-gray'],strokeDash=[5,5]).transform_filter('datum.year==1999').encode(
    x=alt.X(
        "year:Q"
    )
)
fc=base.mark_rule(color=colors['eco-gray'],strokeDash=[3,2]).transform_filter('datum.year==2001').encode(
    x=alt.X(
        "year:Q",
        sort=[],
        axis=alt.Axis(
            labelExpr='datum.label == 1999 ? "r" : datum.label')
    )
)
t_a=eq.mark_text(text='Earthquake',color=colors['eco-gray'],align='right',xOffset=-5,yOffset=-70)
t_b=eq.mark_text(text='Earthquake',color=colors['eco-gray'],align='right',xOffset=-5,yOffset=65)
tf=fc.mark_text(text='Financial crisis',color=colors['eco-gray'],align='left',xOffset=5,yOffset=65)
eq2=base2.mark_rule(color=colors['eco-gray'],strokeDash=[5,5]).transform_filter('datum.d==0')
t2_a=eq2.mark_text(text='Earthquake',color=colors['eco-gray'],align='right',xOffset=-5,yOffset=-70)
t2_b=eq2.mark_text(text='Earthquake',color=colors['eco-gray'],align='right',xOffset=-5,yOffset=65)
label1=alt.Chart(pd.DataFrame([{'x':2002,'y':100,'t':'Türkiye'}])).mark_text(
    color=colors['eco-dot'],align='left').encode(
    x='x:Q',y='y:Q',text='t:N'
)
label3=alt.Chart(pd.DataFrame([{'x':1998.5,'y':105,'t':'Synthetic control'}])).mark_text(
    color=colors['eco-mid-blue'],align='right').encode(
    x='x:Q',y='y:Q',text='t:N'
)
xrule=alt.Chart(pd.DataFrame(
    [{'x':1994,'y':100},{'x':1998,'y':100},{'x':1998,'y':70}]
)).mark_line(color=colors['eco-gray'],strokeWidth=0.3).encode(x='x:Q',y="y:Q")
xrule2=alt.Chart(pd.DataFrame(
    [{'x':-5,'y':100},{'x':-1,'y':100},{'x':-1,'y':70}]
)).mark_line(color=colors['eco-gray'],strokeWidth=0.3).encode(x='x:Q',y="y:Q")

layer1 = (
    ((line+line2+eq+t_a+fc+tf+label1+label3+xrule).properties(height=200, width=250))
).transform_filter('datum.side=="right"')
layer2 = (
    ((line3+line4+eq+t_b+fc+tf).properties(height=200, width=250))
).transform_filter('datum.side=="right"')
layer3 = (
    ((area1+line5+line6+eq2+t2_a+xrule2).properties(height=200, width=250))
).transform_filter('datum.side=="left"')
layer4 = (
    ((area2+line7+line8+eq2+t2_b).properties(height=200, width=250))
).transform_filter('datum.side=="left"')
left=alt.vconcat(
    layer3, layer4
)
right=alt.vconcat(
    layer1, layer2
)
chart=alt.hconcat(left,right).configure_view(stroke=None)
chart.save("visualisation/" + f + ".json")
chart.save("visualisation/" + f + ".svg")
chart.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
chart

/opt/conda/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():
/opt/conda/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():
/opt/conda/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.HConcatChart(...)